Semantic Cache Documentation: https://python.langchain.com/docs/integrations/llms/llm_caching#redis-cache
LLM Documentation: https://python.langchain.com/docs/integrations/llms/azure_openai


In [1]:
%pip install openai langchain redis tiktoken python-dotenv --user

In [2]:
import openai
import redis
import os
import langchain
from os import environ
from langchain.llms import AzureOpenAI
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.globals import set_llm_cache
from langchain.cache import RedisSemanticCache
from dotenv import load_dotenv
import time

load_dotenv()
openai_api_type="azure"
openai_api_version="2023-05-15"
openai_api_base=os.getenv("OPENAI_API_BASE")
openai_api_key=os.getenv("OPENAI_API_KEY")
# I want to pass these in explicitly
os.unsetenv("OPENAI_API_BASE")
os.unsetenv("OPENAI_API_KEY")

# make sure you have an LLM deployed in your Azure Open AI account. In this example, I used the GPT 3.5 turbo instruct model. My deployment was named "gpt35instruct".
llm = AzureOpenAI(
    azure_deployment="resdissemcache",
    model_name="gpt-35-turbo",
    openai_api_base=openai_api_base,
    openai_api_key=openai_api_key,
    api_version=openai_api_version,    
)

# make sure you have an embeddings model deployed in your Azure Open AI account. In this example, I used the text embedding ada 002 model. My deployment was named "textembedding".
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    base_url=openai_api_base,
    api_version=openai_api_version,
    api_key=openai_api_key,
)

REDIS_ENDPOINT = os.getenv('REDIS_ENDPOINT') # must include port at the end. e.g. "redisdemo.eastus.redisenterprise.cache.azure.net:10000"
REDIS_PASSWORD = os.getenv('REDIS_PASSWORD')

# create a connection string for the Redis Vector Store. Uses Redis-py format: https://redis-py.readthedocs.io/en/stable/connections.html#redis.Redis.from_url
# This example assumes TLS is enabled. If not, use "redis://" instead of "rediss://
redis_url = "rediss://:" + REDIS_PASSWORD + "@"+ REDIS_ENDPOINT

# set up the semantic cache for your llm
set_llm_cache(RedisSemanticCache(redis_url = redis_url, embedding=embeddings, score_threshold=.05))

#note: you can use score_threshold to change how sensitive the semantic cache is. The lower the score, the less likely it is to use a cached result.

C:\Users\pahuber\AppData\Roaming\Python\Python312\site-packages\langchain_community\llms\openai.py:872: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://eastus.api.cognitive.microsoft.com/ to https://eastus.api.cognitive.microsoft.com/openai.
  warnings.warn(
C:\Users\pahuber\AppData\Roaming\Python\Python312\site-packages\langchain_community\llms\openai.py:879: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
C:\Users\pahuber\AppData\Roaming\Python\Python312\site-packages\langchain_community\llms\openai.py:887: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example

In [8]:
%%time
response = llm("Please write a poem about cute puppies")
print(response)

. Puppies, puppies everywhere, Everywhere I look. Furry little faces And wagging tails Book Review: How to Teach Physics to Your Dog by Chad Orzel. Nerdy physics professor, Chad Orzel, has written a book that attempts to teach quantum physics to his dog Emmy.

The book is an introduction to the principles that underlie the world of the very small, from atoms and photons, to electrons and quarks. This is a deeply subversive book about the relationship between dogs and humans and what "training" a dog means.

The author argues that dogs are not, as we believe, obedient animals, but rather they are emotionally complex and highly manipulative creatures, adept at controlling their owners and getting what they want/5.

A Dog’s Life is the story of a stray dog named Squirrel and her life in the streets and wilderness. Along her journey, she meets many other dogs and forms bonds with them, both good and bad. Throughout the book, Squirrel is in search of food, shelter, and companionship, but sh

In [5]:
# calculate tokens required
import tiktoken
encoding = tiktoken.get_encoding('cl100k_base')
response_tokens = len(encoding.encode(response))
query_tokens = len(encoding.encode("Please write a poem about cute puppies"))
print(response_tokens)
print(query_tokens)

256
7


In [9]:
%%time
response = llm("Tell me a poem about puppies")
print(response)

. Puppies, puppies everywhere, Everywhere I look. Furry little faces And wagging tails Book Review: How to Teach Physics to Your Dog by Chad Orzel. Nerdy physics professor, Chad Orzel, has written a book that attempts to teach quantum physics to his dog Emmy.

The book is an introduction to the principles that underlie the world of the very small, from atoms and photons, to electrons and quarks. This is a deeply subversive book about the relationship between dogs and humans and what "training" a dog means.

The author argues that dogs are not, as we believe, obedient animals, but rather they are emotionally complex and highly manipulative creatures, adept at controlling their owners and getting what they want/5.

A Dog’s Life is the story of a stray dog named Squirrel and her life in the streets and wilderness. Along her journey, she meets many other dogs and forms bonds with them, both good and bad. Throughout the book, Squirrel is in search of food, shelter, and companionship, but sh